In [1]:
import os
import pandas as pd
import json
from extractor.gnps import GnpsCacher
from extractor.gnps import GnpsTasks
from extractor.gnps import GnpsInchiScore
from extractor.mgfs import MgfFiles
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import Descriptors
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Avalon import pyAvalonTools
from rdkit.Chem.Draw import rdDepictor
import matchms
from pathlib import Path
from extractor.manufactured.datadirs import *

os.chdir("..")

compounds = pd.read_csv(GENERATED_DIR_SUMMARY / "Compounds joined.tsv", sep="\t").set_index("Id")
counts = pd.read_csv(GENERATED_DIR_SUMMARY / "Counts.tsv", sep="\t").set_index("Column")
task_ids_file = GENERATED_DIR_GNPS_TASKS / "Gnps task ids.json"
with open(task_ids_file) as task_ids_data:
    task_ids = set(json.load(task_ids_data))
ts = GnpsTasks(GENERATED_DIR_SUMMARY / "Fetched/", task_ids)
ts.load()


In [2]:
ts.task_id_to_attempt_df()

,Min peaks,Max Δ mass
Task id,,
d083418735184ae2a422696e62d614fb,6,0.02
1c3cbef069874546aca7cca96cb01a05,6,0.10
4dc38f3e2d0a4536b2ddb91d6526fca7,6,10.00
5b2a64759d634312bc87cb0d592f72d0,6,25.00
fa610649898b44d78758d5158aae709e,6,50.00
c0f2f7fe2bd643399674fd4749ad23fe,6,100.00
4d44377069ce4d81a064e7bcaa318435,6,250.00
340e24739a4149dd8dc809dcdcbb0d0b,6,500.00
453f810a7fbe484cb5a047650403bee8,6,inf


In [3]:
selected = compounds.columns.map(lambda c: ("peaks ≥ 6; Δ mass ≤ 0.1" in c) or ("peaks ≥ 6; Δ mass ≤ 10.0" in c))
filtered = compounds.loc[:, ["InChI GNPS; peaks ≥ 6; Δ mass ≤ 0.1", "Smiles GNPS; peaks ≥ 6; Δ mass ≤ 0.1", "Standard InChI GNPS; peaks ≥ 6; Δ mass ≤ 0.1", "InChI GNPS; peaks ≥ 6; Δ mass ≤ 10.0", "Smiles GNPS; peaks ≥ 6; Δ mass ≤ 10.0", "Standard InChI GNPS; peaks ≥ 6; Δ mass ≤ 10.0"]]
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None)
filtered

,InChI GNPS; peaks ≥ 6; Δ mass ≤ 0.1,Smiles GNPS; peaks ≥ 6; Δ mass ≤ 0.1,Standard InChI GNPS; peaks ≥ 6; Δ mass ≤ 0.1,InChI GNPS; peaks ≥ 6; Δ mass ≤ 10.0,Smiles GNPS; peaks ≥ 6; Δ mass ≤ 10.0,Standard InChI GNPS; peaks ≥ 6; Δ mass ≤ 10.0
Id,,,,,,
1,1S/C15H24N2O2/c18-12-4-5-16-8-10-6-11(14(16)7-...,C1C[C@@H]2[C@H]3C[C@@H](CN2C(=O)C1)[C@@H]4C[C@...,InChI=1S/C15H24N2O2/c18-12-4-5-16-8-10-6-11(14...,1S/C15H24N2O2/c18-12-4-5-16-8-10-6-11(14(16)7-...,C1C[C@@H]2[C@H]3C[C@@H](CN2C(=O)C1)[C@@H]4C[C@...,InChI=1S/C15H24N2O2/c18-12-4-5-16-8-10-6-11(14...
2,NaN,NaN,NaN,NaN,NaN,NaN
3,1S/C22H35NO2/c1-14-7-6-9-17(23(14)3)11-12-19-1...,CC1CCCC(N1C)C=CC2C3CCCCC3CC4C2C(OC4=O)C,InChI=1S/C22H35NO2/c1-14-7-6-9-17(23(14)3)11-1...,"""""""InChI=1S/C23H38O4/c1-13(10-21(26)27)16-4-5-...",C[C@@H]([C@H]1CC[C@]2([H])[C@]1(C)CC[C@@]3([H]...,InChI=1S/C23H38O4/c1-13(10-21(26)27)16-4-5-17-...
4,1S/C20H27NO3/c1-8-3-19-6-10-15-18(2)4-9(22)5-2...,CC12CN3C4CC56CC(=C)C7C(O)C5C3C(CC(O)C1)(C24)C6C7O,InChI=1S/C20H27NO3/c1-8-3-19-6-10-15-18(2)4-9(...,"""""""InChI=1S/C24H38O3/c1-15(4-9-22(26)27)19-7-8...",C[C@@H]([C@H]1CC[C@]2([H])[C@]1(C)CC[C@@]3([H]...,InChI=1S/C24H38O3/c1-15(4-9-22(26)27)19-7-8-20...
5,"""""""InChI=1S/C11H11NO4/c1-2-16-11(13)7-6-9-4-3-...",CCOC(=O)C=Cc1cccc([N+](=O)[O-])c1,InChI=1S/C11H11NO4/c1-2-16-11(13)7-6-9-4-3-5-1...,InChI=1S/C12H18O2/c1-8(2)11-6-5-9(3)12(7-11)14...,O=C(OC1C(=CCC(C(=C)C)C1)C)C,InChI=1S/C12H18O2/c1-8(2)11-6-5-9(3)12(7-11)14...
6,InChI=1S/C16H19N3/c1-19-10-3-2-6-16(19)14-7-8-...,N1=CC=CC(=C1)C=2N=CC(=CC2)C3N(C)CCCC3,InChI=1S/C16H19N3/c1-19-10-3-2-6-16(19)14-7-8-...,InChI=1S/C15H15NO3/c1-15(2)7-6-10-13(19-15)11-...,O=C1NC=2C=CC(OC)=CC2C=3OC(C=CC13)(C)C,InChI=1S/C15H15NO3/c1-15(2)7-6-10-13(19-15)11-...
7,InChI=1S/C22H26O7/c1-6-7-22-10-16(25-4)17(23)2...,NaN,InChI=1S/C22H26O7/c1-6-7-22-10-16(25-4)17(23)2...,InChI=1S/C26H31NO4/c1-14-11-20(29-5)25-17(9-8-...,COC1=C2C(C(C3=C(OC)C=C(OC)C4=C3CC(C)NC4C)=C(C)...,InChI=1S/C26H31NO4/c1-14-11-20(29-5)25-17(9-8-...
8,InChI=1S/C15H11N5/c16-15-18-8-6-12(20-15)14-13...,NC1=NC=CC(C2=NC=CC3=C2NC4=CC=CC=C34)=N1,InChI=1S/C15H11N5/c16-15-18-8-6-12(20-15)14-13...,InChI=1S/C15H11N5/c16-15-18-8-6-12(20-15)14-13...,NC1=NC=CC(C2=NC=CC3=C2NC4=CC=CC=C34)=N1,InChI=1S/C15H11N5/c16-15-18-8-6-12(20-15)14-13...
9,1S/C10H12O4/c1-2-3-7(11)6-4-9(13)10(14)5-8(6)1...,CCCC(=O)C1=CC(O)=C(O)C=C1O,InChI=1S/C10H12O4/c1-2-3-7(11)6-4-9(13)10(14)5...,1S/C10H12O4/c1-2-3-7(11)6-4-9(13)10(14)5-8(6)1...,CCCC(=O)C1=CC(O)=C(O)C=C1O,InChI=1S/C10H12O4/c1-2-3-7(11)6-4-9(13)10(14)5...
